# 🎯 Taller Integrador - SQL + Python
## Proyecto Final: Dashboard de Métricas Olist

### Contexto del Negocio

El CEO de Olist necesita un **dashboard de métricas** para la reunión trimestral. Como Analista de Datos, debes:

1. Extraer datos con SQL
2. Procesarlos con Python
3. Crear visualizaciones ejecutivas

### Métricas Solicitadas
- 📈 Evolución de ventas mensual
- 🏆 Top 5 categorías de productos
- 🗺️ Distribución geográfica de clientes
- ⭐ Análisis de satisfacción del cliente
- 🚚 Rendimiento de entregas

---

## ⚙️ Configuración Inicial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from datetime import datetime

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

conn = sqlite3.connect('olist.db')

print("✅ Configuración completada")
print(f"   Fecha del reporte: {datetime.now().strftime('%Y-%m-%d')}")

---

## 📈 Métrica 1: Evolución de Ventas Mensual

In [ ]:
query_ventas = """
SELECT 
    strftime('%Y-%m', o.order_purchase_timestamp) as mes,
    COUNT(DISTINCT o.order_id) as total_pedidos,
    ROUND(SUM(oi.price), 2) as ingresos,
    ROUND(AVG(oi.price), 2) as ticket_promedio
FROM orders o
INNER JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY strftime('%Y-%m', o.order_purchase_timestamp)
ORDER BY mes
"""

df_ventas = pd.read_sql_query(query_ventas, conn)
df_ventas['mes'] = pd.to_datetime(df_ventas['mes'])
print(f"✅ Datos extraídos: {len(df_ventas)} meses")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

axes[0].plot(df_ventas['mes'], df_ventas['ingresos']/1000, 
             marker='o', linewidth=2, markersize=6, color='#2ecc71')
axes[0].fill_between(df_ventas['mes'], df_ventas['ingresos']/1000, alpha=0.3, color='#2ecc71')
axes[0].set_title('Evolución de Ingresos Mensuales', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Ingresos (Miles R$)')
axes[0].grid(True, alpha=0.3)

axes[1].bar(df_ventas['mes'], df_ventas['total_pedidos'], color='#3498db', alpha=0.7)
axes[1].set_title('Número de Pedidos por Mes', fontsize=16, fontweight='bold')
axes[1].set_ylabel('Número de Pedidos')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 RESUMEN: Ingresos totales R$ {df_ventas['ingresos'].sum():,.2f}")

---

## 🏆 Métrica 2: Top Categorías de Productos

**Tu turno:** Extrae y visualiza las top 10 categorías

In [ ]:
# Tu consulta SQL aquí
query_categorias = """
-- Escribe tu consulta
"""

<details>
<summary>💡 Ver solución</summary>

```python
query_categorias = """
SELECT 
    COALESCE(ct.product_category_name_english, 'Sin categoría') as categoria,
    COUNT(*) as items_vendidos,
    ROUND(SUM(oi.price), 2) as ingresos
FROM order_items oi
INNER JOIN orders o ON oi.order_id = o.order_id
INNER JOIN products p ON oi.product_id = p.product_id
LEFT JOIN category_translation ct ON p.product_category_name = ct.product_category_name
WHERE o.order_status = 'delivered'
GROUP BY ct.product_category_name_english
ORDER BY ingresos DESC
LIMIT 10
"""

df_categorias = pd.read_sql_query(query_categorias, conn)

fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(df_categorias)))
ax.barh(df_categorias['categoria'], df_categorias['ingresos']/1000, color=colors[::-1])
ax.set_xlabel('Ingresos (Miles R$)')
ax.set_title('Top 10 Categorías por Ingresos', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()
```
</details>

---

## 🗺️ Métrica 3: Distribución Geográfica

**Tu turno:** Analiza los clientes por estado

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
query_geo = """
SELECT 
    c.customer_state as estado,
    COUNT(DISTINCT c.customer_id) as total_clientes,
    COUNT(DISTINCT o.order_id) as total_pedidos,
    ROUND(SUM(oi.price), 2) as ingresos
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
ORDER BY ingresos DESC
LIMIT 10
"""

df_geo = pd.read_sql_query(query_geo, conn)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].pie(df_geo['ingresos'], labels=df_geo['estado'], autopct='%1.1f%%')
axes[0].set_title('Distribución de Ingresos por Estado')

axes[1].barh(df_geo['estado'], df_geo['total_clientes'], color='#9b59b6')
axes[1].set_xlabel('Número de Clientes')
axes[1].set_title('Clientes por Estado')

plt.tight_layout()
plt.show()
```
</details>

---

## ⭐ Métrica 4: Satisfacción del Cliente

**Tu turno:** Analiza la distribución de reviews y su relación con entregas tardías

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
query_reviews = """
SELECT 
    r.review_score,
    COUNT(*) as cantidad,
    CASE 
        WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 'Tardío'
        ELSE 'A tiempo'
    END as tipo_entrega
FROM order_reviews r
INNER JOIN orders o ON r.order_id = o.order_id
WHERE o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
GROUP BY r.review_score, tipo_entrega
ORDER BY r.review_score
"""

df_reviews = pd.read_sql_query(query_reviews, conn)

df_pivot = df_reviews.pivot(index='review_score', columns='tipo_entrega', values='cantidad').fillna(0)

df_pivot.plot(kind='bar', figsize=(10, 6), color=['#e74c3c', '#2ecc71'])
plt.title('Calificaciones según Tipo de Entrega', fontsize=16, fontweight='bold')
plt.xlabel('Review Score')
plt.ylabel('Cantidad')
plt.legend(title='Entrega')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()
```
</details>

---

## 🚚 Métrica 5: Rendimiento de Entregas

**Tu turno:** Calcula el tiempo promedio de entrega por mes

In [ ]:
# Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```python
query_entregas = """
SELECT 
    strftime('%Y-%m', order_purchase_timestamp) as mes,
    ROUND(AVG(julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp)), 1) as dias_promedio,
    SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as pct_tardios
FROM orders
WHERE order_status = 'delivered'
    AND order_delivered_customer_date IS NOT NULL
GROUP BY strftime('%Y-%m', order_purchase_timestamp)
ORDER BY mes
"""

df_entregas = pd.read_sql_query(query_entregas, conn)
df_entregas['mes'] = pd.to_datetime(df_entregas['mes'])

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

axes[0].plot(df_entregas['mes'], df_entregas['dias_promedio'], marker='o', color='#3498db')
axes[0].set_title('Tiempo Promedio de Entrega (días)', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Días')
axes[0].axhline(y=df_entregas['dias_promedio'].mean(), color='red', linestyle='--', label='Promedio global')
axes[0].legend()

axes[1].bar(df_entregas['mes'], df_entregas['pct_tardios'], color='#e74c3c', alpha=0.7)
axes[1].set_title('Porcentaje de Entregas Tardías', fontsize=16, fontweight='bold')
axes[1].set_ylabel('% Tardíos')

plt.tight_layout()
plt.show()
```
</details>

---

## 📋 Resumen Ejecutivo Final

**Tu turno:** Crea un resumen con las métricas clave

In [ ]:
query_resumen = """
SELECT 
    (SELECT COUNT(DISTINCT order_id) FROM orders WHERE order_status = 'delivered') as pedidos_entregados,
    (SELECT ROUND(SUM(price), 2) FROM order_items oi 
        INNER JOIN orders o ON oi.order_id = o.order_id WHERE o.order_status = 'delivered') as ingresos_totales,
    (SELECT COUNT(DISTINCT customer_id) FROM customers) as total_clientes,
    (SELECT COUNT(DISTINCT seller_id) FROM sellers) as total_vendedores,
    (SELECT ROUND(AVG(review_score), 2) FROM order_reviews) as review_promedio
"""

resumen = pd.read_sql_query(query_resumen, conn).iloc[0]

print("="*60)
print("          📊 RESUMEN EJECUTIVO OLIST")
print("="*60)
print(f"\n  🛒 Pedidos entregados:    {resumen['pedidos_entregados']:,}")
print(f"  💰 Ingresos totales:      R$ {resumen['ingresos_totales']:,.2f}")
print(f"  👥 Total clientes:        {resumen['total_clientes']:,}")
print(f"  🏪 Total vendedores:      {resumen['total_vendedores']:,}")
print(f"  ⭐ Review promedio:        {resumen['review_promedio']}/5.0")
print("\n" + "="*60)

In [ ]:
# Cerrar conexión
conn.close()
print("\n✅ Análisis completado. Conexión cerrada.")

---

## 🎉 ¡Felicitaciones!

Has completado el taller integrador combinando:
- **SQL** para extracción y agregación de datos
- **Python/Pandas** para procesamiento
- **Matplotlib/Seaborn** para visualización

### Próximos pasos sugeridos:
1. Exportar las visualizaciones para una presentación
2. Crear un dashboard interactivo con Streamlit o Dash
3. Construir un modelo predictivo de satisfacción del cliente